<div style="text-align: left; font-family: 'charter'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
# PDF Downloader

import os  # for file and directory manipulation
import random  # to generate random numbers
import time  # to manage time and take breaks in the script
import requests  # to make HTTP requests to web servers
from selenium import webdriver  # for automating web browsers
from selenium.webdriver.common.by import By  # to locate elements on a webpage
from selenium.webdriver.support.ui import WebDriverWait  # to wait until certain conditions are met on a webpage.
from selenium.webdriver.support import expected_conditions as EC  # to define expected conditions
from selenium.common.exceptions import StaleElementReferenceException  # To handle exceptions related to elements on the webpage that are no longer available.


# Extracting Tables (and data cleaning)

import pdfplumber  # for extracting text and metadata from PDF files
import pandas as pd  # for data manipulation and analysis
import os  # for interacting with the operating system
import unicodedata  # for manipulating Unicode data
import re  # for regular expressions operations
from datetime import datetime  # for working with dates and times
import locale  # for locale-specific formatting of numbers, dates, and currencies


# SQL tables

import psycopg2  # for interacting with PostgreSQL databases
from sqlalchemy import create_engine, text  # for creating and executing SQL queries using SQLAlchemy

In [2]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import unidecode

In [3]:
# Obtener variables de entorno
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Verificar si todas las variables de entorno están definidas
if not all([host, user, password]):
    raise ValueError("Faltan algunas variables de entorno (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Crear la cadena de conexión
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Crear el motor de SQLAlchemy
engine = create_engine(connection_string)

# Definir la consulta SQL para importar datos
sector = 'gdp'  # Define el sector según tus necesidades
query = f"SELECT * FROM {sector}_monthly_growth_rates"

# Importar los datos a un DataFrame de pandas
df = pd.read_sql(query, engine)

# Ahora 'df' contiene los datos importados desde la base de datos
df.head() # Muestra las primeras filas del DataFrame para verificar

,year,id_ns,date,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,...,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,2013,01,2013-01-04,5.9,5.3,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,5.9,5.3,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(df['sep_2011'].dtype)

float64


# Cargando desde excel

In [5]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\2012\ns-03-2012.csv'

# Carrega o arquivo CSV para um DataFrame
df = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
df.head(5)

,SECTORES ECONÓMICOS,2010,Unnamed: 2,Unnamed: 3,2011,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,ECONOMIC SECTORS
0,NaN,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
3,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
4,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock


# Data clean-up

### Auxiliares

In [6]:
def remove_rare_characters_first_row(texto):
    texto = re.sub(r'\s*-\s*', '-', texto)  # Remueve espacios alrededor de guiones
    texto = re.sub(r'[^a-zA-Z0-9\s-]', '', texto)  # Remueve caracteres raros excepto letras, dígitos y guiones
    return texto

def remove_rare_characters(texto):
    return re.sub(r'[^a-zA-Z\s]', '', texto)

def remove_tildes(texto):
    return ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))

### Común

In [7]:
# 0.
def drop_nan_rows(df):
    df = df.dropna(how='all')
    return df

# 1. 
def drop_nan_columns(df):
    return df.dropna(axis=1, how='all')

# 2.
def swap_first_second_row(df):
    temp = df.iloc[0, 0]
    df.iloc[0, 0] = df.iloc[1, 0]
    df.iloc[1, 0] = temp

    temp = df.iloc[0, -1]
    df.iloc[0, -1] = df.iloc[1, -1]
    df.iloc[1, -1] = temp
    return df

# 8. 
def reset_index(df):
    df.reset_index(drop=True, inplace=True)
    return df

# 5.
def remove_digit_slash(df):
    # Aplica la función de reemplazo a la primera columna y a las dos últimas columnas
    df.iloc[:, [0, -2, -1]] = df.iloc[:, [0, -2, -1]].apply(lambda x: x.str.replace(r'\d+/', '', regex=True))
    return df

# 9. AUX (ROBUSTO)

def separate_text_digits(df):
    for index, row in df.iterrows():
        if any(char.isdigit() for char in str(row.iloc[-2])) and any(char.isalpha() for char in str(row.iloc[-2])):
            if pd.isnull(row.iloc[-1]):
                df.loc[index, df.columns[-1]] = ''.join(filter(lambda x: x.isalpha() or x == ' ', str(row.iloc[-2])))
                df.loc[index, df.columns[-2]] = ''.join(filter(lambda x: not (x.isalpha() or x == ' '), str(row.iloc[-2])))
            
            # Check if comma or dot is used as decimal separator
            if ',' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split(',')
            elif '.' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split('.')
            else:
                # If neither comma nor dot found, assume no decimal part
                split_values = [str(row.iloc[-2]), '']
                
            cleaned_integer = ''.join(filter(lambda x: x.isdigit() or x == '-', split_values[0]))
            cleaned_decimal = ''.join(filter(lambda x: x.isdigit(), split_values[1]))
            if cleaned_decimal:
                # Use comma as decimal separator
                cleaned_numeric = cleaned_integer + ',' + cleaned_decimal
            else:
                cleaned_numeric = cleaned_integer
            df.loc[index, df.columns[-2]] = cleaned_numeric
    return df


# 4. 
def extract_years(df):
    year_columns = [col for col in df.columns if re.match(r'\b\d{4}\b', col)]
    #print("Años (4 dígitos) extraídos:")
    #print(year_columns)
    return year_columns

# 6. 
def first_row_columns(df):
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df

# 15.
def clean_columns_values(df):
    df.columns = df.columns.str.lower()
    # Only normalize string column names
    df.columns = [unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8') if isinstance(col, str) else col for col in df.columns]
    df.columns = df.columns.str.replace(' ', '_').str.replace('ano', 'year').str.replace('-', '_')
    
    text_columns = df.select_dtypes(include='object').columns
    for col in df.columns:
        df.loc[:, col] = df[col].apply(lambda x: remove_tildes(x) if isinstance(x, str) else x)
        df.loc[:, col] = df[col].apply(lambda x: str(x).replace(',', '.') if isinstance(x, (int, float, str)) else x)
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].str.lower()
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].str.lower()
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].apply(remove_rare_characters)
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].apply(remove_rare_characters)
    return df

# 16.
def convertir_float(df):
    excluded_columns = ['sectores_economicos', 'economic_sectors']
    columns_to_convert = [col for col in df.columns if col not in excluded_columns]
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
    return df

# 15.
def relocate_last_column(df):
    last_column = df.pop(df.columns[-1])
    df.insert(1, last_column.name, last_column)
    return df

### Exclusiva Tabla 1

In [8]:
# ATIPIC LAST COLUMNS
def relocate_last_columns(df):
    if not pd.isna(df.iloc[1, -1]):
        # Create a new column with NaN
        new_column = 'col_' + ''.join(map(str, np.random.randint(1, 5, size=1)))
        df[new_column] = np.nan
        
        # Get 'ECONOMIC SECTORS' and relocate
        insert_value_1 = df.iloc[0, -2]
        # Convert the value to string before assignment
        insert_value_1 = str(insert_value_1)
        # Ensure the dtype of the last column is object (string) to accommodate string values
        df.iloc[:, -1] = df.iloc[:, -1].astype('object')
        df.iloc[0, -1] = insert_value_1
        
        # NaN first obs
        df.iloc[0,-2] = np.nan
    return df

# Extraer meses

def get_months_sublist_list(df, year_columns):
    first_row = df.iloc[0]
    # Initialize the list of sublists
    months_sublist_list = []

    # Initialize the current sublist
    months_sublist = []

    # Iterate over the elements of the first row
    for item in first_row:
        # Check if the item meets the requirements
        if len(str(item)) == 3:
            months_sublist.append(item)
        elif '-' in item or str(item) == 'year':
            months_sublist.append(item)
            months_sublist_list.append(months_sublist)
            months_sublist = []

    # Add the last sublist if it's not empty
    if months_sublist:
        months_sublist_list.append(months_sublist)

    new_elements = []

    # Check if year_columns is not empty
    if year_columns:
        for i, year in enumerate(year_columns):
            # Check if index i is valid for quarters_sublist_list
            if i < len(months_sublist_list):
                for element in months_sublist_list[i]:
                    new_elements.append(f"{year}_{element}")
                    
    two_first_elements = df.iloc[0][:2].tolist()

    # Ensure that the two_first_elements are added if they are not in new_elements
    for index in range(len(two_first_elements) - 1, -1, -1):
        if two_first_elements[index] not in new_elements:
            new_elements.insert(0, two_first_elements[index])

    # Ensure that the length of new_elements matches the number of columns in df
    while len(new_elements) < len(df.columns):
        new_elements.append(None)

    temp_df = pd.DataFrame([new_elements], columns=df.columns)
    df.iloc[0] = temp_df.iloc[0]

    return df


def find_year_column(df):
    # List to store the found years
    found_years = []

    # Iterating over the column names of the DataFrame
    for column in df.columns:
        # Checking if the column name is a year (4 digits)
        if column.isdigit() and len(column) == 4:
            found_years.append(column)

    # If more than one year is found, do nothing
    if len(found_years) > 1:
        pass
    # If exactly one year is found, implement additional code
    elif len(found_years) == 1:
        # Getting the name of the found year
        year_name = found_years[0]
        print("The name of the column representing the year is:", year_name)

        # Getting the first row of the DataFrame
        first_row = df.iloc[0]

        # Searching for the first column containing the word "year" or some hyphen-separated expression
        column_contains_year = first_row[first_row.astype(str).str.contains(r'\byear\b')]

        if not column_contains_year.empty:
            # Getting the name of the first column containing 'year' or some hyphen-separated expression in the first row
            column_contains_year_name = column_contains_year.index[0]
            print("The name of the first column containing 'year' or some hyphen-separated expression in the first row is:", column_contains_year_name)

            # Getting the indices of the columns
            column_contains_year_index = df.columns.get_loc(column_contains_year_name)
            year_name_index = df.columns.get_loc(year_name)
            print("The index of the column containing 'year' is:", column_contains_year_index)
            print("The index of the column representing the year is:", year_name_index)

            # Checking if the column representing the year is to the right or to the left of column_contains_year
            if column_contains_year_index < year_name_index:
                print("The year column is to the right of the column containing 'year'.")
                # Adding one to the year
                new_year = str(int(year_name) - 1)
                # Renaming the column containing 'year' with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            elif column_contains_year_index > year_name_index:
                print("The year column is to the left of the column containing 'year'.")
                # Subtracting one from the year
                new_year = str(int(year_name) + 1)
                # Renaming the year column with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            else:
                print("The year column is in the same position as the column containing 'year'.")
        else:
            print("No columns containing 'year' were found in the first row.")
    # If no year is found, print a message
    else:
        print("No years were found in the column names.")
    
    return df


#

def clean_first_row(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            if isinstance(df.at[0, col], str):
                df.at[0, col] = df.at[0, col].lower()  # Convertir a minúsculas solo si es un objeto
                df.at[0, col] = remove_tildes(df.at[0, col])
                df.at[0, col] = remove_rare_characters_first_row(df.at[0, col])
                # Reemplazar 'ano' por 'year'
                df.at[0, col] = df.at[0, col].replace('ano', 'year')

    return df


def intercambiar_valores(df):
    # Verificar si hay al menos dos columnas en el DataFrame
    if len(df.columns) < 2:
        print("El DataFrame tiene menos de dos columnas. No se pueden intercambiar valores.")
        return df

    # Verificar si hay valores NaN en la última columna
    if df.iloc[:, -1].isnull().any():
        # Obtener índice de filas con NaN en la última columna
        last_column_rows_nan = df[df.iloc[:, -1].isnull()].index

        # Iterar sobre las filas con NaN en la última columna
        for idx in last_column_rows_nan:
            # Verificar si el índice está dentro del rango de las columnas
            if -2 >= -len(df.columns):
                # Intercambiar los valores de la última columna y la penúltima columna
                df.iloc[idx, -1], df.iloc[idx, -2] = df.iloc[idx, -2], df.iloc[idx, -1]
            else:
                print(f"Índice fuera de rango para la fila {idx}. No se pueden intercambiar valores.")

    return df



def replace_var_perc_first_column(df):
    # Regular expression to search for "Var. %" or "Var.%"
    regex = re.compile(r'Var\. ?%')

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        # Convert the value in the first column to a string
        value = str(row.iloc[0])

        # Check if the value matches the regular expression
        if regex.search(value):
            # Replace only the characters that match the regular expression
            df.at[index, df.columns[0]] = regex.sub("variacion porcentual", value)
    
    return df


# 8.
number_moving_average = 'three' # Keep a space at the end

def replace_number_moving_average(df):
    for index, row in df.iterrows():
        # Buscar la expresión regular en la penúltima o última columna
        if pd.notnull(row.iloc[-1]) and re.search(r'(\d\s*-)', str(row.iloc[-1])):
            df.at[index, df.columns[-1]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-1]))
        #elif pd.notnull(row.iloc[-2]) and re.search(r'(\d\s*-)', str(row.iloc[-2])):
        #   df.at[index, df.columns[-2]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-2]))
    return df


# 7.
def replace_var_perc_last_columns(df):
    # Expresión regular para buscar "Var. %" o "Var.%"
    regex = re.compile(r'(Var\. ?%)(.*)')

    # Iterar sobre las filas del dataframe
    for index, row in df.iterrows():
        # Verificar si el valor en la penúltima columna es una cadena no nula
        if isinstance(row.iloc[-2], str) and regex.search(row.iloc[-2]):
            # Realizar el reemplazo al final del valor de la penúltima columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-2])
            df.at[index, df.columns[-2]] = replaced_text.strip()
        
        # Verificar si el valor en la última columna es una cadena no nula
        if isinstance(row.iloc[-1], str) and regex.search(row.iloc[-1]):
            # Realizar el reemplazo al final del valor de la última columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-1])
            df.at[index, df.columns[-1]] = replaced_text.strip()
    
    return df

# Función para buscar y reemplazar en la segunda fila del DataFrame
def replace_first_dot(df):
    second_row = df.iloc[1]  # Segunda fila del DataFrame
    
    # Verificar si al menos una observación cumple con el patrón
    if any(isinstance(cell, str) and re.match(r'^\w+\.\s?\w+', cell) for cell in second_row):
        for col in df.columns:
            if isinstance(second_row[col], str):  # Verificar si el valor es una cadena
                if re.match(r'^\w+\.\s?\w+', second_row[col]):  # Verificar si cumple con el patrón Xxx.Xxx o Xxx. Xxx.
                    df.at[1, col] = re.sub(r'(\w+)\.(\s?\w+)', r'\1-\2', second_row[col], count=1)  # Reemplazar solo el primer punto
    return df

def drop_rare_caracter_row(df):
    # Buscar el caracter solitario "}" en cada fila y obtener un booleano para cada fila
    rare_caracter_row = df.apply(lambda row: '}' in row.values, axis=1)
    
    # Filtrar el DataFrame para eliminar las filas con el caracter solitario "}"
    df = df[~rare_caracter_row]
    
    return df

def split_column_by_pattern(df):
    # Iteramos sobre las columnas del dataframe
    for col in df.columns:
        # Verificamos si la segunda fila de la columna contiene el patrón
        if re.match(r'^[A-Z][a-z]+\.?\s[A-Z][a-z]+\.?$', str(df.iloc[1][col])):
            # Realizamos el split de la columna usando como criterio el espacio
            split_values = df[col].str.split(expand=True)
            # Guardamos los primeros valores en la columna original
            df[col] = split_values[0]
            # Guardamos los segundos valores en una nueva columna con el sufijo "_split"
            new_col_name = col + '_split'
            df.insert(df.columns.get_loc(col) + 1, new_col_name, split_values[1])
    return df

$\Large{\color{blue}{ns\_2014\_07}}$

se: sectores económicos

In [9]:
def swap_nan_se(df):
    # Check if the first observation of the first column is NaN
    if pd.isna(df.iloc[0, 0]) and df.iloc[0, 1] == "SECTORES ECONÓMICOS":
        # Create a temporary copy of the values
        column_1_value = df.iloc[0, 1]
        # Swap values in the original row
        df.iloc[0, 0] = column_1_value
        df.iloc[0, 1] = np.nan
        # Drop the second column
        df = df.drop(df.columns[1], axis=1)
    return df

$\Large{\color{blue}{set \ by \ sep}}$

In [10]:
def replace_set_sep(df):
    # Get the column names of the DataFrame
    columns = df.columns
    
    # Iterate over the columns
    for column in columns:
        # Check if the column contains the expression 'set'
        if 'set' in column:
            # Replace 'set' with 'sep' in the column name
            new_column = column.replace('set', 'sep')
            # Rename the column in the DataFrame
            df.rename(columns={column: new_column}, inplace=True)
    
    return df

$\Large{\color{blue}{strip}}$

In [11]:
def spaces_se_es(df):
    # Aplicar strip a las columnas 'sectores_economicos' y 'economic_sectors'
    df['sectores_economicos'] = df['sectores_economicos'].str.strip()
    df['economic_sectors'] = df['economic_sectors'].str.strip()
    return df

$\Large{\color{blue}{Replace \ other \ services}}$

In [12]:
def replace_services(df):
    # Verificamos si se encuentran los valores 'otros servicios' y 'other services'
    if ('servicios' in df['sectores_economicos'].values) and ('services' in df['economic_sectors'].values):
        # Reemplazamos los valores
        df['sectores_economicos'].replace({'servicios': 'otros servicios'}, inplace=True)
        df['economic_sectors'].replace({'services': 'other services'}, inplace=True)
    return df

# Exclusivas para tablas entregadas por el BCRP

In [13]:
# 1.

In [123]:
def limpiar_nombres_columnas(df):
    # Eliminar tildes y convertir a minúsculas los nombres de las columnas
    df.columns = df.columns.str.lower()
    # Only normalize string column names
    df.columns = [unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8') if isinstance(col, str) else col for col in df.columns]
    return df

In [124]:
# 2.

In [125]:
def ajustar_nombres_columnas(df):
    # Comprobar que la primera observación de la primera columna sea NaN
    if pd.isna(df.iloc[0, 0]) and pd.isna(df.iloc[0, -1]):
        # Verificar los nombres de las columnas
        if "sectores economicos" in df.columns[0] and "economic sectors" in df.columns[-1]:
            # Reemplazar NaN por los nombres de las columnas correspondientes
            df.iloc[0, 0] = "sectores economicos"
            df.iloc[0, -1] = "economic sectors"
    return df

In [126]:
# 3.

In [127]:
def redondear_valores(df, decimales=1):
    # Iterar sobre todas las columnas del DataFrame
    for col in df.columns:
        # Verificar si la columna es de tipo float
        if df[col].dtype == 'float64':
            # Redondear los valores de la columna al número de decimales especificado
            df[col] = df[col].round(decimales)
    return df

# Importando csv

In [97]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-07-2010.csv'

# Carrega o arquivo CSV para um DataFrame
df = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
df.head(5)

,SECTORES ECONÓMICOS,2008,Unnamed: 2,2009,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,ECONOMIC SECTORS
0,NaN,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Dic.,Año,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,5.069764433,7.210717423,-1.362736901,4.961511313,9.053442947,2.084984438,6.076785707,-4.993202172,1.186796531,3.59598778,2.321085235,0.841434297,2.680355717,2.151199111,2.29442892,Agriculture and Livestock
3,Agrícola,2.385281486,7.359216634,-5.921608447,4.079454569,11.04216854,0.617198964,7.181175762,-9.058779406,0.605152181,0.789347888,-0.200716926,-1.934663178,3.346639124,0.839868542,0.906907626,Agriculture
4,Pecuario,7.3351341,6.015017884,4.369167155,6.081122075,6.229132171,5.384615198,3.422684997,3.933606145,2.062474381,7.123086064,5.059012425,4.018531731,1.916971915,3.645845395,4.404016489,Livestock


# Test

In [35]:
df = clean_columns_values(df)
df

KeyError: 'economic_sectors'

In [71]:
df.columns[-1]

' ECONOMIC SECTORS'

In [72]:

df

,sectores economicos,2008,unnamed: 2,2009,unnamed: 4,unnamed: 5,unnamed: 6,unnamed: 7,unnamed: 8,unnamed: 9,unnamed: 10,unnamed: 11,unnamed: 12,unnamed: 13,unnamed: 14,unnamed: 15,economic sectors
0,sectores economicos,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Dic.,Año,economic sectors
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,5.069764433,7.210717423,-1.362736901,4.961511313,9.053442947,2.084984438,6.076785707,-4.993202172,1.186796531,3.59598778,2.321085235,0.841434297,2.680355717,2.151199111,2.29442892,Agriculture and Livestock
3,Agrícola,2.385281486,7.359216634,-5.921608447,4.079454569,11.04216854,0.617198964,7.181175762,-9.058779406,0.605152181,0.789347888,-0.200716926,-1.934663178,3.346639124,0.839868542,0.906907626,Agriculture
4,Pecuario,7.3351341,6.015017884,4.369167155,6.081122075,6.229132171,5.384615198,3.422684997,3.933606145,2.062474381,7.123086064,5.059012425,4.018531731,1.916971915,3.645845395,4.404016489,Livestock
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,7.138628686,6.2409014,-17.08769926,-9.682901554,-14.96651196,-16.07978883,33.39536997,-8.486666862,-4.657815447,-12.31409818,-0.17589986,-18.58283649,-4.65972081,-21.74103479,-7.665398291,Fishing
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e hidrocarburos,3.403146608,7.578566372,10.66432769,-1.91761051,2.537349812,2.770910067,1.629445666,-2.164533419,0.263919404,1.070180006,-1.063379482,-0.503948657,-1.099608998,-5.739073029,0.3971959,Mining and fuel
9,Minería metálica,1.401377938,7.284922073,7.828201097,-3.727631514,-1.110021486,1.406032845,-0.476518238,-3.919674919,-1.79091859,-1.213750127,-3.106084841,-1.202239739,-1.561669739,-6.835635906,-1.413992054,Metals


In [89]:
df = limpiar_nombres_columnas(df)
df = ajustar_nombres_columnas(df)
df = drop_rare_caracter_row(df)
df = drop_nan_rows(df)
df = drop_nan_columns(df)
df = reset_index(df)
df = remove_digit_slash(df)
df = replace_var_perc_first_column(df)
df = replace_var_perc_last_columns(df)
df = replace_number_moving_average(df)
df = relocate_last_column(df)
df = clean_first_row(df)
df = find_year_column(df)
year_columns = extract_years(df)
df = get_months_sublist_list(df, year_columns)
df = first_row_columns(df)
df = clean_columns_values(df)
df = convertir_float(df)
df = replace_set_sep(df)
df = spaces_se_es(df)
df = replace_services(df)
df = redondear_valores(df, decimales=1)
df

,sectores_economicos,economic_sectors,2008_dic,2008_year,2009_ene,2009_feb,2009_mar,2009_abr,2009_may,2009_jun,2009_jul,2009_ago,2009_sep,2009_oct,2009_nov,2009_dic,2009_year
1,agropecuario,agriculture and livestock,5.1,7.2,-1.4,5.0,9.1,2.1,6.1,-5.0,1.2,3.6,2.3,0.8,2.7,2.2,2.3
2,agricola,agriculture,2.4,7.4,-5.9,4.1,11.0,0.6,7.2,-9.1,0.6,0.8,-0.2,-1.9,3.3,0.8,0.9
3,pecuario,livestock,7.3,6.0,4.4,6.1,6.2,5.4,3.4,3.9,2.1,7.1,5.1,4.0,1.9,3.6,4.4
4,pesca,fishing,7.1,6.2,-17.1,-9.7,-15.0,-16.1,33.4,-8.5,-4.7,-12.3,-0.2,-18.6,-4.7,-21.7,-7.7
5,mineria e hidrocarburos,mining and fuel,3.4,7.6,10.7,-1.9,2.5,2.8,1.6,-2.2,0.3,1.1,-1.1,-0.5,-1.1,-5.7,0.4
6,mineria metalica,metals,1.4,7.3,7.8,-3.7,-1.1,1.4,-0.5,-3.9,-1.8,-1.2,-3.1,-1.2,-1.6,-6.8,-1.4
7,hidrocarburos,fuel,21.9,10.3,36.5,15.0,40.2,15.3,21.4,13.6,17.3,22.0,16.1,4.8,2.6,2.7,16.1
8,manufactura,manufacturing,4.5,8.8,-0.5,-6.7,-5.5,-13.6,-8.0,-13.1,-12.2,-10.4,-8.3,-5.9,-2.6,1.6,-7.2
9,de recursos primarios,based on raw materials,10.3,7.6,10.4,3.1,-1.5,-9.7,23.2,-6.3,-7.2,10.6,0.1,-11.6,-3.5,-4.4,0.0
10,no primaria,nonprimary,2.8,8.9,-2.5,-8.4,-6.2,-14.4,-14.1,-14.6,-13.1,-13.5,-9.7,-4.9,-2.5,3.0,-8.5


In [90]:
import os
import pandas as pd

In [131]:
# Define la ruta base donde están las carpetas por año
base_path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1'

# Diccionario para almacenar los DataFrames limpios
dfs = {}

# Iterar sobre cada año (carpeta dentro de base_path)
for year_folder in os.listdir(base_path):
    year_folder_path = os.path.join(base_path, year_folder)
    
    # Verificar si es una carpeta
    if os.path.isdir(year_folder_path):
        # Iterar sobre cada archivo CSV dentro de la carpeta del año
        for csv_file in os.listdir(year_folder_path):
            if csv_file.endswith('.csv'):
                csv_file_path = os.path.join(year_folder_path, csv_file)
                
                # Cargar el archivo CSV como DataFrame
                df = pd.read_csv(csv_file_path, delimiter=';')
                df = limpiar_nombres_columnas(df)
                df = ajustar_nombres_columnas(df)
                df = drop_rare_caracter_row(df)
                df = drop_nan_rows(df)
                df = drop_nan_columns(df)
                df = reset_index(df)
                df = remove_digit_slash(df)
                df = replace_var_perc_first_column(df)
                df = replace_var_perc_last_columns(df)
                df = replace_number_moving_average(df)
                df = relocate_last_column(df)
                df = clean_first_row(df)
                df = find_year_column(df)
                year_columns = extract_years(df)
                df = get_months_sublist_list(df, year_columns)
                df = first_row_columns(df)
                df = clean_columns_values(df)
                df = convertir_float(df)
                df = replace_set_sep(df)
                df = spaces_se_es(df)
                df = replace_services(df)
                df = redondear_valores(df, decimales=1)
                
                # Guardar el DataFrame limpio en el diccionario usando el nombre del archivo como clave
                dfs[csv_file] = df

In [133]:
dfs.keys()

dict_keys(['ns-03-2010.csv', 'ns-04-2010.csv', 'ns-07-2010.csv', 'ns-08-2010.csv', 'ns-11-2010.csv', 'ns-15-2010.csv', 'ns-19-2010.csv', 'ns-23-2010.csv', 'ns-28-2010.csv', 'ns-31-2010.csv', 'ns-36-2010.csv', 'ns-40-2010.csv', 'ns-44-2010.csv', 'ns-48-2010.csv', 'ns-03-2011.csv', 'ns-07-2011.csv', 'ns-08-2011.csv', 'ns-11-2011.csv', 'ns-16-2011.csv', 'ns-19-2011.csv', 'ns-23-2011.csv', 'ns-24-2011.csv', 'ns-28-2011.csv', 'ns-31-2011.csv', 'ns-36-2011.csv', 'ns-40-2011.csv', 'ns-44-2011.csv', 'ns-49-2011.csv', 'ns-03-2012.csv', 'ns-08-2012.csv', 'ns-12-2012.csv', 'ns-15-2012.csv', 'ns-19-2012.csv', 'ns-24-2012.csv', 'ns-25-2012.csv', 'ns-28-2012.csv', 'ns-31-2012.csv', 'ns-36-2012.csv', 'ns-40-2012.csv', 'ns-45-2012.csv', 'ns-49-2012.csv'])

In [134]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import locale
from tkinter import Tk, messagebox
from sqlalchemy import create_engine

# Establecer la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Diccionario para almacenar los DataFrames generados
old_dataframes_dict_1 = {}

# Ruta del archivo de registro de carpetas procesadas
registro_path = 'dataframes_record/old_carpetas_procesadas_1.txt'

# Función para corregir los nombres de los meses
def corregir_nombre_mes(mes):
    meses_mapping = {
        'setiembre': 'septiembre',
        # Agrega más mapeos si es necesario para otros nombres de meses
    }
    return meses_mapping.get(mes, mes)

def registrar_carpeta_procesada(carpeta, num_archivos_procesados):
    with open(registro_path, 'a') as file:
        file.write(f"{carpeta}:{num_archivos_procesados}\n")

def carpeta_procesada(carpeta):
    if not os.path.exists(registro_path):
        return False
    with open(registro_path, 'r') as file:
        for line in file:
            if line.startswith(carpeta):
                return True
    return False

def obtener_fecha(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    fecha = pd.read_sql(query, engine)
    return fecha.iloc[0, 0] if not fecha.empty else None

def procesar_archivo_csv(csv_path, engine):
    old_tables_dict_1 = {}  # Diccionario local para cada archivo CSV
    table_counter = 1

    filename = os.path.basename(csv_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No se encontraron coincidencias para id_ns y year en el nombre del archivo:", filename)
        return None, None, None, None

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    nombre_dataframe = f"{new_filename}_{table_counter}"
    old_tables_dict_1[nombre_dataframe] = df.copy()

    # Aplicar las funciones de limpieza a una copia del DataFrame
    df_clean = df.copy()

    # Funciones de limpieza (ajustar según sea necesario)
    df_clean = limpiar_nombres_columnas(df_clean)
    df_clean = ajustar_nombres_columnas(df_clean)
    df_clean = drop_rare_caracter_row(df_clean)
    df_clean = drop_nan_rows(df_clean)
    df_clean = drop_nan_columns(df_clean)
    df_clean = reset_index(df_clean)
    df_clean = remove_digit_slash(df_clean)
    df_clean = replace_var_perc_first_column(df_clean)
    df_clean = replace_var_perc_last_columns(df_clean)
    df_clean = replace_number_moving_average(df_clean)
    df_clean = relocate_last_column(df_clean)
    df_clean = clean_first_row(df_clean)
    df_clean = find_year_column(df_clean)
    year_columns = extract_years(df_clean)
    df_clean = get_months_sublist_list(df_clean, year_columns)
    df_clean = first_row_columns(df_clean)
    df_clean = clean_columns_values(df_clean)
    df_clean = convertir_float(df_clean)
    df_clean = replace_set_sep(df_clean)
    df_clean = spaces_se_es(df_clean)
    df_clean = replace_services(df_clean)
    df_clean = redondear_valores(df_clean, decimales=1)

    # Añadir la columna 'year' al DataFrame limpio
    df_clean.insert(0, 'year', year)
    
    # Añadir la columna 'id_ns' al DataFrame limpio
    df_clean.insert(1, 'id_ns', id_ns)
    
    # Obtener la fecha correspondiente de la base de datos
    fecha = obtener_fecha(df_clean, engine)
    if fecha:
        # Añadir la columna 'date' al DataFrame limpio
        df_clean.insert(2, 'date', fecha)
    else:
        print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
    
    # Almacenar DataFrame limpio en old_dataframes_dict_1
    old_dataframes_dict_1[nombre_dataframe] = df_clean

    return id_ns, year, old_tables_dict_1

def procesar_carpeta(carpeta, engine):
    print(f"Procesando la carpeta {os.path.basename(carpeta)}")
    csv_files = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith('.csv')]

    num_csv_procesados = 0
    num_dataframes_generados = 0

    table_counter = 1  # Inicializar el contador de tabla aquí
    old_tables_dict_1 = {}  # Declarar old_tables_dict_1 fuera del bucle principal
    
    for csv_file in csv_files:
        id_ns, year, tables_dict_temp = procesar_archivo_csv(csv_file, engine)

        if tables_dict_temp:
            for nombre_df, df in tables_dict_temp.items():
                nombre_archivo = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                nombre_df = f"{nombre_archivo}_{table_counter}"
                
                # Almacenar DataFrame sin procesar en old_tables_dict_1
                old_tables_dict_1[nombre_df] = df.copy()
                
                # Procesar y limpiar el DataFrame
                df_clean = df.copy()
                
                # Aplicar las funciones de limpieza a una copia del DataFrame
                df_clean = limpiar_nombres_columnas(df_clean)
                df_clean = ajustar_nombres_columnas(df_clean)
                df_clean = drop_rare_caracter_row(df_clean)
                df_clean = drop_nan_rows(df_clean)
                df_clean = drop_nan_columns(df_clean)
                df_clean = reset_index(df_clean)
                df_clean = remove_digit_slash(df_clean)
                df_clean = replace_var_perc_first_column(df_clean)
                df_clean = replace_var_perc_last_columns(df_clean)
                df_clean = replace_number_moving_average(df_clean)
                df_clean = relocate_last_column(df_clean)
                df_clean = clean_first_row(df_clean)
                df_clean = find_year_column(df_clean)
                year_columns = extract_years(df_clean)
                df_clean = get_months_sublist_list(df_clean, year_columns)
                df_clean = first_row_columns(df_clean)
                df_clean = clean_columns_values(df_clean)
                df_clean = convertir_float(df_clean)
                df_clean = replace_set_sep(df_clean)
                df_clean = spaces_se_es(df_clean)
                df_clean = replace_services(df_clean)
                df_clean = redondear_valores(df_clean, decimales=1)
                
                # Añadir la columna 'year' al DataFrame limpio
                df_clean.insert(0, 'year', year)
                
                # Añadir la columna 'id_ns' al DataFrame limpio
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Obtener la fecha correspondiente de la base de datos
                fecha = obtener_fecha(df_clean, engine)
                if fecha:
                    # Añadir la columna 'date' al DataFrame limpio
                    df_clean.insert(2, 'date', fecha)
                else:
                    print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
                
                # Almacenar DataFrame limpio en old_dataframes_dict_1
                old_dataframes_dict_1[nombre_df] = df_clean

                print(f'  {table_counter}. El dataframe generado para el archivo {csv_file} es: {nombre_df}')
                num_dataframes_generados += 1
                table_counter += 1  # Incrementar el contador de tabla aquí
        
        num_csv_procesados += 1  # Incrementar el número de CSVs procesados por cada archivo en la carpeta

    return num_csv_procesados, num_dataframes_generados, old_tables_dict_1

def procesar_carpetas():
    base_folder = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1'
    carpetas = [os.path.join(base_folder, d) for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_1 = {}  # Inicializar old_tables_dict_1 aquí
    
    for carpeta in carpetas:
        if carpeta_procesada(carpeta):
            print(f"La carpeta {carpeta} ya ha sido procesada.")
            continue
        
        num_csv_procesados, num_dataframes_generados, tables_dict_temp = procesar_carpeta(carpeta, engine)
        
        # Actualizar old_tables_dict_1 con los valores devueltos de procesar_carpeta()
        old_tables_dict_1.update(tables_dict_temp)
        
        registrar_carpeta_procesada(carpeta, num_csv_procesados)

        # Preguntar al usuario si desea continuar con la siguiente carpeta
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Para asegurar que la ventana esté en primer plano
        
        mensaje = f"Se han generado {num_dataframes_generados} dataframes en la carpeta {carpeta}. ¿Deseas continuar con la siguiente carpeta?"
        continuar = messagebox.askyesno("Continuar", mensaje)
        root.destroy()

        if not continuar:
            print("Procesamiento detenido por el usuario.")
            break  # Romper el bucle for si el usuario decide no continuar

    print("Procesamiento completado para todas las carpetas.")  # Add a message to indicate completion

    return old_tables_dict_1  # Devolver old_tables_dict_1 al final de la función

if __name__ == "__main__":
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')
    port = 5432
    database = 'gdp_revisions_datasets'

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)

    old_tables_dict_1 = procesar_carpetas()  # Capturar el valor devuelto de procesar_carpetas()


Procesando la carpeta 2010
  1. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\2010\ns-03-2010.csv es: ns_03_2010_1
  2. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\2010\ns-04-2010.csv es: ns_04_2010_2
  3. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\2010\ns-07-2010.csv es: ns_07_2010_3
  4. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\2010\ns-08-2010.csv es: ns_08_2010_4
  5. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\2010\ns-11-2010.csv es: ns_11_2010_5
  6. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\2010\ns-15-2010.csv 

In [137]:
old_tables_dict_1['ns_03_2010_1']

,SECTORES ECONÓMICOS,2008,Unnamed: 2,Unnamed: 3,2009,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,ECONOMIC SECTORS
0,NaN,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.- Nov.,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,7.706801858,5.069764433,7.210650283,-1.223271137,4.935758635,8.153814568,1.79234973,6.526257114,-4.679446813,1.483373545,3.562378071,2.481326266,0.26648885,2.311604569,2.263036485,Agriculture and Livestock
3,Agrícola,4.61540992,2.407522314,7.368974133,-6.901089175,2.921391825,10.3531738,1.156047355,7.308782745,-8.368026213,1.415387017,3.826190032,0.22793874,-3.493134666,2.479854709,1.048939729,Agriculture
4,Pecuario,11.44718178,8.249792898,6.95827139,5.195829383,7.156430119,4.995833259,4.06896777,2.746345233,3.598827015,0.964657334,7.600173181,4.932533769,4.568195228,2.129376786,4.314059267,Livestock
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,-8.605707412,5.208824361,6.192465383,-17.75124246,-11.55071925,-17.55629162,-17.08010683,31.28581338,-12.5007266,-8.273029295,-9.439100396,0.144202127,-18.21019566,-5.67,-7.648897888,Fishing
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e hidrocarburos,12.38408563,3.450423862,7.57836306,10.66432818,-1.917631048,2.537351335,2.769155094,1.628694357,-2.163640477,0.318233431,1.12140623,-1.053127179,-0.504,-1.100405502,1.011424131,Mining and fuel
9,Minería metálica,10.90010606,1.401377938,7.277484849,7.828201097,-3.727631514,-1.110021486,1.406032845,-0.476518238,-3.919674919,-1.752656183,-1.177029505,-3.056297098,-1.202239739,-1.58,-0.879030541,Metals


In [139]:
old_dataframes_dict_1['ns_03_2010_1']

,year,id_ns,date,sectores_economicos,economic_sectors,2008_nov,2008_dic,2008_year,2009_ene,2009_feb,2009_mar,2009_abr,2009_may,2009_jun,2009_jul,2009_ago,2009_sep,2009_oct,2009_nov,2009_ene_nov
1,2010,03,2010-01-22,agropecuario,agriculture and livestock,7.7,5.1,7.2,-1.2,4.9,8.2,1.8,6.5,-4.7,1.5,3.6,2.5,0.3,2.3,2.3
2,2010,03,2010-01-22,agricola,agriculture,4.6,2.4,7.4,-6.9,2.9,10.4,1.2,7.3,-8.4,1.4,3.8,0.2,-3.5,2.5,1.0
3,2010,03,2010-01-22,pecuario,livestock,11.4,8.2,7.0,5.2,7.2,5.0,4.1,2.7,3.6,1.0,7.6,4.9,4.6,2.1,4.3
4,2010,03,2010-01-22,pesca,fishing,-8.6,5.2,6.2,-17.8,-11.6,-17.6,-17.1,31.3,-12.5,-8.3,-9.4,0.1,-18.2,-5.7,-7.6
5,2010,03,2010-01-22,mineria e hidrocarburos,mining and fuel,12.4,3.5,7.6,10.7,-1.9,2.5,2.8,1.6,-2.2,0.3,1.1,-1.1,-0.5,-1.1,1.0
6,2010,03,2010-01-22,mineria metalica,metals,10.9,1.4,7.3,7.8,-3.7,-1.1,1.4,-0.5,-3.9,-1.8,-1.2,-3.1,-1.2,-1.6,-0.9
7,2010,03,2010-01-22,hidrocarburos,fuel,25.3,21.9,10.3,36.5,15.0,40.2,15.3,21.4,13.6,17.3,21.9,16.1,4.8,2.6,17.6
8,2010,03,2010-01-22,manufactura,manufacturing,1.4,4.1,8.7,-0.6,-6.9,-5.8,-14.0,-7.5,-13.6,-12.8,-10.9,-8.7,-6.5,-2.5,-8.2
9,2010,03,2010-01-22,de recursos primarios,based on raw materials,4.2,10.3,7.6,10.6,2.9,-2.2,-10.3,28.3,-7.5,-8.2,10.1,0.1,-12.1,-3.4,0.6
10,2010,03,2010-01-22,no primaria,nonprimary,0.8,2.8,8.9,-2.6,-8.6,-6.5,-14.7,-14.4,-14.9,-13.5,-13.9,-10.1,-5.5,-2.3,-9.8


# Libraries

In [205]:
import roman

# Cargando desde excel

In [360]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-04-2010.csv'

# Carrega o arquivo CSV para um DataFrame
df_clean = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
df_clean.head(5)

,SECTORES ECONÓMICOS,2007,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,2008,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2009,Unnamed: 12,Unnamed: 13,ECONOMIC SECTORS
0,NaN,I,II,III,IV,TOTAL,I,II,III,IV,TOTAL,I,II,III,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,8.354957256,2.284101494,-1.448867011,5.012741027,3.283438973,3.893475841,8.013706611,9.434138024,7.10408736,7.210650283,4.140600429,1.348842262,2.469252877,Agriculture and Livestock
3,Agrícola,5.967716367,1.132601373,-4.542544163,7.099641476,1.976145502,4.633263727,8.37903945,10.43667636,5.135850235,7.368974133,2.883966742,0.350115368,1.037047925,Agriculture
4,Pecuario,10.72572697,5.139037185,2.765906141,2.909444829,5.27291312,2.945428669,7.205260893,8.177815384,9.456773529,6.95827139,5.77107241,3.590255218,4.292209419,Livestock


### Exclusiva Tabla 2

In [322]:
def clean_first_row(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            if isinstance(df.at[0, col], str):
                df.at[0, col] = df.at[0, col].lower()  # Convertir a minúsculas solo si es un objeto
                df.at[0, col] = remove_tildes(df.at[0, col])
                df.at[0, col] = remove_rare_characters_first_row(df.at[0, col])
                # Reemplazar 'ano' por 'year'
                df.at[0, col] = df.at[0, col].replace('ano', 'year')

    return df

# 2.
def separate_years(df):
    df = df.copy()  # Se crea una copia del DataFrame para evitar SettingWithCopyWarning
    if isinstance(df.iloc[0, -2], str) and len(df.iloc[0, -2].split()) == 2:
        years = df.iloc[0, -2].split()
        if all(len(year) == 4 for year in years):
            segundo_anio = years[1]
            df.iloc[0, -2] = years[0]
            df.insert(len(df.columns) - 1, 'new_column', [segundo_anio] + [None] * (len(df) - 1))
    return df

# 3.
def find_roman_numerals(text):
    pattern = r'\b(?:I{1,3}|IV|V|VI{0,3}|IX|X)\b'
    matches = re.findall(pattern, text)
    return matches

def relocate_roman_numerals(df):
    numeros_romanos = find_roman_numerals(df.iloc[2, -1])
    if numeros_romanos:
        original_text = df.iloc[2, -1]
        for roman_numeral in numeros_romanos:
            original_text = original_text.replace(roman_numeral, '').strip()
        df.iloc[2, -1] = original_text
        df.at[2, 'new_column'] = ', '.join(numeros_romanos)
        df.iloc[2, -1] = np.nan
    return df

# 4.
def extract_mixed_values(df):
    df = df.copy()  # Se crea una copia del DataFrame para evitar SettingWithCopyWarning
    regex_pattern = r'(-?\d+,\d [a-zA-Z\s]+)'
    for index, row in df.iterrows():
        antepenultima_obs = row.iloc[-3]
        penultima_obs = row.iloc[-2]

        if isinstance(antepenultima_obs, str) and pd.notnull(antepenultima_obs):
            match = re.search(regex_pattern, antepenultima_obs)
            if match:
                parte_extraida = match.group(0)
                if pd.isna(penultima_obs) or pd.isnull(penultima_obs):
                    df.iloc[index, -2] = parte_extraida
                    antepenultima_obs = re.sub(regex_pattern, '', antepenultima_obs).strip()
                    df.iloc[index, -3] = antepenultima_obs
    return df

# 5.
def replace_first_row_nan(df):
    for col in df.columns:
        if pd.isna(df.iloc[0][col]):
            df.iloc[0, df.columns.get_loc(col)] = col
    return df

# 11. 
def split_values(df):
    columna_a_expandir = df.columns[-3]
    nuevas_columnas = df[columna_a_expandir].str.split(expand=True)
    nuevas_columnas.columns = [f'{columna_a_expandir}_{i+1}' for i in range(nuevas_columnas.shape[1])]
    posicion_insercion = len(df.columns) - 2
    for col in reversed(nuevas_columnas.columns):
        df.insert(posicion_insercion, col, nuevas_columnas[col])
    df.drop(columns=[columna_a_expandir], inplace=True)
    return df


# 13.
def roman_arabic(df):
    primera_fila = df.iloc[0]
    def convert_roman_number(numero):
        try:
            return str(roman.fromRoman(numero))
        except roman.InvalidRomanNumeralError:
            return numero

    primera_fila_convertida = []
    for valor in primera_fila:
        if isinstance(valor, str) and not pd.isna(valor):
            primera_fila_convertida.append(convert_roman_number(valor))
        else:
            primera_fila_convertida.append(valor)

    df.iloc[0] = primera_fila_convertida
    return df

# 14.
def fix_duplicates(df):
    fila_segunda = df.iloc[0].copy()
    prev_num = None
    first_one_index = None

    for i, num in enumerate(fila_segunda):
        try:
            num = int(num)
            prev_num = int(prev_num) if prev_num is not None else None

            if num == prev_num:
                if num == 1:
                    if first_one_index is None:
                        first_one_index = i - 1
                    next_num = int(fila_segunda[i - 1]) + 1
                    for j in range(i, len(fila_segunda)):
                        if fila_segunda.iloc[j].isdigit():
                            fila_segunda.iloc[j] = str(next_num)
                            next_num += 1
                elif i - 1 >= 0:
                    fila_segunda.iloc[i] = str(int(fila_segunda.iloc[i - 1]) + 1)

            prev_num = num
        except ValueError:
            pass

    df.iloc[0] = fila_segunda
    return df

## More

In [350]:
def get_quarters_sublist_list(df, year_columns):
    first_row = df.iloc[0]
    # Initialize the list of sublists
    quarters_sublist_list = []

    # Initialize the current sublist
    quarters_sublist = []

    # Iterate over the elements of the first row
    for item in first_row:
        # Check if the item meets the requirements
        if len(str(item)) == 1:
            quarters_sublist.append(item)
        elif str(item) == 'year':
            quarters_sublist.append(item)
            quarters_sublist_list.append(quarters_sublist)
            quarters_sublist = []

    # Add the last sublist if it's not empty
    if quarters_sublist:
        quarters_sublist_list.append(quarters_sublist)

    new_elements = []

    # Check if year_columns is not empty
    if year_columns:
        for i, year in enumerate(year_columns):
            # Check if index i is valid for quarters_sublist_list
            if i < len(quarters_sublist_list):
                for element in quarters_sublist_list[i]:
                    new_elements.append(f"{year}_{element}")

    two_first_elements = df.iloc[0][:2].tolist()

    # Ensure that the two_first_elements are added if they are not in new_elements
    for index in range(len(two_first_elements) - 1, -1, -1):
        if two_first_elements[index] not in new_elements:
            new_elements.insert(0, two_first_elements[index])

    # Ensure that the length of new_elements matches the number of columns in df
    while len(new_elements) < len(df.columns):
        new_elements.append(None)

    temp_df = pd.DataFrame([new_elements], columns=df.columns)
    df.iloc[0] = temp_df.iloc[0]

    return df

$\Large{\color{blue}{ns\_2016\_20}}$

In [210]:
def drop_nan_row(df):
    if df.iloc[0].isnull().all():
        df = df.drop(index=0)
        df.reset_index(drop=True, inplace=True)
    return df

# Total by year

In [358]:
def replace_total_with_year(df):
    # Reemplazar 'TOTAL' con 'AÑO' en la primera fila
    df.iloc[0] = df.iloc[0].apply(lambda x: 'AÑO' if "TOTAL" in str(x) else x)
    return df

In [359]:
df_clean = replace_total_with_year(df_clean)
df_clean

,SECTORES ECONÓMICOS,2007,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,2008,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2009,Unnamed: 12,Unnamed: 13,ECONOMIC SECTORS
0,NaN,I,II,III,IV,AÑO,I,II,III,IV,AÑO,I,II,III,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,8.354957256,2.284101494,-1.448867011,5.012741027,3.283438973,3.893475841,8.013706611,9.434138024,7.10408736,7.210650283,4.140600429,1.348842262,2.469252877,Agriculture and Livestock
3,Agrícola,5.967716367,1.132601373,-4.542544163,7.099641476,1.976145502,4.633263727,8.37903945,10.43667636,5.135850235,7.368974133,2.883966742,0.350115368,1.037047925,Agriculture
4,Pecuario,10.72572697,5.139037185,2.765906141,2.909444829,5.27291312,2.945428669,7.205260893,8.177815384,9.456773529,6.95827139,5.77107241,3.590255218,4.292209419,Livestock
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,14.49614142,9.430250169,-5.308493851,5.628280175,6.870039683,6.566013316,4.367043048,14.97827389,2.520467501,6.192465383,-15.69585673,-0.642292795,-6.038907036,Fishing
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e Hidrocarburos,-0.999071169,-1.36310387,5.238654452,7.815027662,2.712675236,6.219059728,10.03406083,7.497659118,6.648478102,7.57836306,3.650044858,0.71104005,0.137196193,Mining and fuel
9,Minería metálica,-2.655105219,-2.891932254,5.487734419,6.687416385,1.677537339,6.696864092,10.62198105,7.225220274,4.841649766,7.277484849,0.867342541,-1.033691986,-1.986258853,Metals


# Test

In [356]:
df_clean

,SECTORES ECONÓMICOS,2007,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,2008,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2009,Unnamed: 12,Unnamed: 13,ECONOMIC SECTORS
0,NaN,I,II,III,IV,TOTAL,I,II,III,IV,TOTAL,I,II,III,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,8.354957256,2.284101494,-1.448867011,5.012741027,3.283438973,3.893475841,8.013706611,9.434138024,7.10408736,7.210650283,4.140600429,1.348842262,2.469252877,Agriculture and Livestock
3,Agrícola,5.967716367,1.132601373,-4.542544163,7.099641476,1.976145502,4.633263727,8.37903945,10.43667636,5.135850235,7.368974133,2.883966742,0.350115368,1.037047925,Agriculture
4,Pecuario,10.72572697,5.139037185,2.765906141,2.909444829,5.27291312,2.945428669,7.205260893,8.177815384,9.456773529,6.95827139,5.77107241,3.590255218,4.292209419,Livestock
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,14.49614142,9.430250169,-5.308493851,5.628280175,6.870039683,6.566013316,4.367043048,14.97827389,2.520467501,6.192465383,-15.69585673,-0.642292795,-6.038907036,Fishing
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e Hidrocarburos,-0.999071169,-1.36310387,5.238654452,7.815027662,2.712675236,6.219059728,10.03406083,7.497659118,6.648478102,7.57836306,3.650044858,0.71104005,0.137196193,Mining and fuel
9,Minería metálica,-2.655105219,-2.891932254,5.487734419,6.687416385,1.677537339,6.696864092,10.62198105,7.225220274,4.841649766,7.277484849,0.867342541,-1.033691986,-1.986258853,Metals


In [361]:
df_clean = replace_total_with_year(df_clean)
df_clean = drop_nan_rows(df_clean)
year_columns = extract_years(df_clean)
df_clean = roman_arabic(df_clean)
df_clean = fix_duplicates(df_clean)
df_clean = relocate_last_column(df_clean)
df_clean = replace_first_row_nan(df_clean)
df_clean = clean_first_row(df_clean)
df_clean = get_quarters_sublist_list(df_clean, year_columns)
df_clean = reset_index(df_clean)
df_clean = first_row_columns(df_clean)
df_clean = clean_columns_values(df_clean)
df_clean = reset_index(df_clean)
df_clean = convertir_float(df_clean)
df_clean = replace_set_sep(df_clean)
df_clean = spaces_se_es(df_clean)
df_clean = replace_services(df_clean)
df_clean = redondear_valores(df_clean, decimales=1)
df_clean

,sectores_economicos,economic_sectors,2007_1,2007_2,2007_3,2007_4,2007_year,2008_1,2008_2,2008_3,2008_4,2008_year,2009_1,2009_2,2009_3
0,agropecuario,agriculture and livestock,8.4,2.3,-1.4,5.0,3.3,3.9,8.0,9.4,7.1,7.2,4.1,1.3,2.5
1,agricola,agriculture,6.0,1.1,-4.5,7.1,2.0,4.6,8.4,10.4,5.1,7.4,2.9,0.4,1.0
2,pecuario,livestock,10.7,5.1,2.8,2.9,5.3,2.9,7.2,8.2,9.5,7.0,5.8,3.6,4.3
3,pesca,fishing,14.5,9.4,-5.3,5.6,6.9,6.6,4.4,15.0,2.5,6.2,-15.7,-0.6,-6.0
4,mineria e hidrocarburos,mining and fuel,-1.0,-1.4,5.2,7.8,2.7,6.2,10.0,7.5,6.6,7.6,3.7,0.7,0.1
5,mineria metalica,metals,-2.7,-2.9,5.5,6.7,1.7,6.7,10.6,7.2,4.8,7.3,0.9,-1.0,-2.0
6,hidrocarburos,fuel,10.6,5.8,0.6,9.7,6.5,2.0,4.9,9.9,22.8,10.3,30.4,16.7,18.4
7,manufactura,manufacturing,9.7,11.2,10.3,11.9,10.8,10.6,11.1,10.0,3.4,8.7,-4.4,-11.7,-10.8
8,de procesamiento de recursos primarios,based on raw materials,0.9,4.6,-2.9,-0.5,0.7,12.5,5.4,3.4,9.4,7.6,3.7,3.2,0.2
9,no primaria,nonprimary,11.2,14.5,13.0,13.3,13.0,10.2,12.3,11.1,2.3,8.9,-5.9,-14.7,-12.5


In [362]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import locale
from tkinter import Tk, messagebox, TOP, YES, NO
from sqlalchemy import create_engine

# Establecer la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Diccionario para almacenar los DataFrames generados
old_dataframes_dict_2 = {}

# Ruta del archivo de registro de carpetas procesadas
registro_path = 'dataframes_record/old_carpetas_procesadas_2.txt'

# Función para corregir los nombres de los meses
def corregir_nombre_mes(mes):
    meses_mapping = {
        'setiembre': 'septiembre',
        # Agrega más mapeos si es necesario para otros nombres de meses
    }
    return meses_mapping.get(mes, mes)

def registrar_carpeta_procesada(carpeta, num_archivos_procesados):
    with open(registro_path, 'a') as file:
        file.write(f"{carpeta}:{num_archivos_procesados}\n")

def carpeta_procesada(carpeta):
    if not os.path.exists(registro_path):
        return False
    with open(registro_path, 'r') as file:
        for line in file:
            if line.startswith(carpeta):
                return True
    return False

def obtener_fecha(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    fecha = pd.read_sql(query, engine)
    return fecha.iloc[0, 0] if not fecha.empty else None

def procesar_archivo_csv(csv_path, engine):
    old_tables_dict_2 = {}  # Diccionario local para cada archivo CSV
    table_counter = 1

    filename = os.path.basename(csv_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No se encontraron coincidencias para id_ns y year en el nombre del archivo:", filename)
        return None, None, None, None

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    nombre_dataframe = f"{new_filename}_{table_counter}"
    old_tables_dict_2[nombre_dataframe] = df.copy()

    # Aplicar las funciones de limpieza a una copia del DataFrame
    df_clean = df.copy()

    # Funciones de limpieza (ajustar según sea necesario)
    df_clean = replace_total_with_year(df_clean)
    df_clean = drop_nan_rows(df_clean)
    year_columns = extract_years(df_clean)
    df_clean = roman_arabic(df_clean)
    df_clean = fix_duplicates(df_clean)
    df_clean = relocate_last_column(df_clean)
    df_clean = replace_first_row_nan(df_clean)
    df_clean = clean_first_row(df_clean)
    df_clean = get_quarters_sublist_list(df_clean, year_columns)
    df_clean = reset_index(df_clean)
    df_clean = first_row_columns(df_clean)
    df_clean = clean_columns_values(df_clean)
    df_clean = reset_index(df_clean)
    df_clean = convertir_float(df_clean)
    df_clean = replace_set_sep(df_clean)
    df_clean = spaces_se_es(df_clean)
    df_clean = replace_services(df_clean)
    df_clean = redondear_valores(df_clean, decimales=1)

    # Añadir la columna 'year' al DataFrame limpio
    df_clean.insert(0, 'year', year)
    
    # Añadir la columna 'id_ns' al DataFrame limpio
    df_clean.insert(1, 'id_ns', id_ns)
    
    # Obtener la fecha correspondiente de la base de datos
    fecha = obtener_fecha(df_clean, engine)
    if fecha:
        # Añadir la columna 'date' al DataFrame limpio
        df_clean.insert(2, 'date', fecha)
    else:
        print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
    
    # Almacenar DataFrame limpio en old_dataframes_dict_2
    old_dataframes_dict_2[nombre_dataframe] = df_clean

    return id_ns, year, old_tables_dict_2

def procesar_carpeta(carpeta, engine):
    print(f"Procesando la carpeta {os.path.basename(carpeta)}")
    csv_files = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith('.csv')]

    num_csv_procesados = 0
    num_dataframes_generados = 0

    table_counter = 1  # Inicializar el contador de tabla aquí
    old_tables_dict_2 = {}  # Declarar old_tables_dict_2 fuera del bucle principal
    
    for csv_file in csv_files:
        id_ns, year, tables_dict_temp = procesar_archivo_csv(csv_file, engine)

        if tables_dict_temp:
            for nombre_df, df in tables_dict_temp.items():
                nombre_archivo = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                nombre_df = f"{nombre_archivo}_{table_counter}"
                
                # Almacenar DataFrame sin procesar en old_tables_dict_2
                old_tables_dict_2[nombre_df] = df.copy()
                
                # Procesar y limpiar el DataFrame
                df_clean = df.copy()
                
                # Aplicar las funciones de limpieza a una copia del DataFrame
                df_clean = replace_total_with_year(df_clean)
                df_clean = drop_nan_rows(df_clean)
                year_columns = extract_years(df_clean)
                df_clean = roman_arabic(df_clean)
                df_clean = fix_duplicates(df_clean)
                df_clean = relocate_last_column(df_clean)
                df_clean = replace_first_row_nan(df_clean)
                df_clean = clean_first_row(df_clean)
                df_clean = get_quarters_sublist_list(df_clean, year_columns)
                df_clean = reset_index(df_clean)
                df_clean = first_row_columns(df_clean)
                df_clean = clean_columns_values(df_clean)
                df_clean = reset_index(df_clean)
                df_clean = convertir_float(df_clean)
                df_clean = replace_set_sep(df_clean)
                df_clean = spaces_se_es(df_clean)
                df_clean = replace_services(df_clean)
                df_clean = redondear_valores(df_clean, decimales=1)
                
                # Añadir la columna 'year' al DataFrame limpio
                df_clean.insert(0, 'year', year)
                
                # Añadir la columna 'id_ns' al DataFrame limpio
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Obtener la fecha correspondiente de la base de datos
                fecha = obtener_fecha(df_clean, engine)
                if fecha:
                    # Añadir la columna 'date' al DataFrame limpio
                    df_clean.insert(2, 'date', fecha)
                else:
                    print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
                
                # Almacenar DataFrame limpio en old_dataframes_dict_2
                old_dataframes_dict_2[nombre_df] = df_clean

                print(f'  {table_counter}. El dataframe generado para el archivo {csv_file} es: {nombre_df}')
                num_dataframes_generados += 1
                table_counter += 1  # Incrementar el contador de tabla aquí
        
        num_csv_procesados += 1  # Incrementar el número de CSVs procesados por cada archivo en la carpeta

    return num_csv_procesados, num_dataframes_generados, old_tables_dict_2

def procesar_carpetas():
    base_folder = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2'
    carpetas = [os.path.join(base_folder, d) for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_2 = {}  # Inicializar old_tables_dict_2 aquí
    
    for carpeta in carpetas:
        if carpeta_procesada(carpeta):
            print(f"La carpeta {carpeta} ya ha sido procesada.")
            continue
        
        num_csv_procesados, num_dataframes_generados, tables_dict_temp = procesar_carpeta(carpeta, engine)
        
        # Actualizar old_tables_dict_2 con los valores devueltos de procesar_carpeta()
        old_tables_dict_2.update(tables_dict_temp)
        
        registrar_carpeta_procesada(carpeta, num_csv_procesados)

        # Preguntar al usuario si desea continuar con la siguiente carpeta
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Para asegurar que la ventana esté en primer plano
        
        mensaje = f"Se han generado {num_dataframes_generados} dataframes en la carpeta {carpeta}. ¿Deseas continuar con la siguiente carpeta?"
        continuar = messagebox.askyesno("Continuar", mensaje)
        root.destroy()

        if not continuar:
            print("Procesamiento detenido por el usuario.")
            break  # Romper el bucle for si el usuario decide no continuar

    print("Procesamiento completado para todas las carpetas.")  # Add a message to indicate completion

    return old_tables_dict_2  # Devolver old_tables_dict_2 al final de la función
    
if __name__ == "__main__":
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')
    port = 5432
    database = 'gdp_revisions_datasets'

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    old_tables_dict_2 = procesar_carpetas()  # Capturar el valor devuelto de procesar_carpetas()

Procesando la carpeta 2010
  1. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-04-2010.csv es: ns_04_2010_1
  2. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-08-2010.csv es: ns_08_2010_2
  3. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-20-2010.csv es: ns_20_2010_3
  4. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-32-2010.csv es: ns_32_2010_4
  5. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-45-2010.csv es: ns_45_2010_5
Procesando la carpeta 2011
  1. El dataframe generado para el archivo C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\t

In [364]:
old_tables_dict_2['ns_04_2010_1']

,SECTORES ECONÓMICOS,2007,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,2008,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2009,Unnamed: 12,Unnamed: 13,ECONOMIC SECTORS
0,NaN,I,II,III,IV,TOTAL,I,II,III,IV,TOTAL,I,II,III,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,8.354957256,2.284101494,-1.448867011,5.012741027,3.283438973,3.893475841,8.013706611,9.434138024,7.10408736,7.210650283,4.140600429,1.348842262,2.469252877,Agriculture and Livestock
3,Agrícola,5.967716367,1.132601373,-4.542544163,7.099641476,1.976145502,4.633263727,8.37903945,10.43667636,5.135850235,7.368974133,2.883966742,0.350115368,1.037047925,Agriculture
4,Pecuario,10.72572697,5.139037185,2.765906141,2.909444829,5.27291312,2.945428669,7.205260893,8.177815384,9.456773529,6.95827139,5.77107241,3.590255218,4.292209419,Livestock
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,14.49614142,9.430250169,-5.308493851,5.628280175,6.870039683,6.566013316,4.367043048,14.97827389,2.520467501,6.192465383,-15.69585673,-0.642292795,-6.038907036,Fishing
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e Hidrocarburos,-0.999071169,-1.36310387,5.238654452,7.815027662,2.712675236,6.219059728,10.03406083,7.497659118,6.648478102,7.57836306,3.650044858,0.71104005,0.137196193,Mining and fuel
9,Minería metálica,-2.655105219,-2.891932254,5.487734419,6.687416385,1.677537339,6.696864092,10.62198105,7.225220274,4.841649766,7.277484849,0.867342541,-1.033691986,-1.986258853,Metals


In [368]:
old_dataframes_dict_2['ns_32_2010_1']

,year,id_ns,date,sectores_economicos,economic_sectors,2008_1,2008_2,2008_3,2008_4,2008_year,2009_1,2009_2,2009_3,2009_4,2009_year,2010_1,2010_2
0,2010,32,2010-08-27,agropecuario,agriculture and livestock,4.0,8.0,9.4,7.1,7.2,4.4,1.1,2.3,1.9,2.3,3.8,5.2
1,2010,32,2010-08-27,agricola,agriculture,4.6,8.4,10.4,5.1,7.4,3.6,-0.2,0.4,0.8,0.9,3.7,4.8
2,2010,32,2010-08-27,pecuario,livestock,2.0,6.3,7.3,8.5,6.0,5.6,4.2,4.7,3.2,4.4,3.5,5.4
3,2010,32,2010-08-27,pesca,fishing,3.9,1.9,21.1,4.6,6.3,-14.2,1.4,-6.1,-14.6,-7.9,-9.9,-12.0
4,2010,32,2010-08-27,mineria e hidrocarburos,mining and fuel,6.1,9.9,7.7,6.9,7.6,3.7,0.5,0.1,-1.6,0.6,0.4,0.8
5,2010,32,2010-08-27,mineria metalica,metals,6.7,10.6,7.2,4.9,7.3,0.9,-1.0,-2.0,-3.2,-1.4,-1.0,-2.0
6,2010,32,2010-08-27,hidrocarburos,fuel,2.0,4.9,9.9,22.8,10.3,30.4,16.7,18.4,3.4,16.1,11.0,22.0
7,2010,32,2010-08-27,manufactura,manufacturing,10.8,11.4,10.5,4.0,9.1,-4.2,-11.6,-10.3,-2.3,-7.2,7.5,17.0
8,2010,32,2010-08-27,de procesamiento de recursos primarios,based on raw materials,12.5,5.4,3.4,9.4,7.6,3.9,2.2,0.7,-6.3,0.0,-5.8,-2.1
9,2010,32,2010-08-27,no primaria,nonprimary,10.2,12.3,11.1,2.3,8.9,-5.7,-14.4,-12.1,-1.6,-8.5,10.1,21.7


# TO SQL

In [1]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\to_sql\old_raw_data_delivered.csv'

# Carrega o arquivo CSV para um DataFrame
old_raw_data_delivered = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
old_raw_data_delivered.head(5)

,year,id_ns,delivered_1,delivered_2
0,1994,1,0,0
1,1994,2,0,0
2,1994,3,0,0
3,1994,4,0,0
4,1994,5,0,0


In [11]:
import os
import shutil
import psycopg2
import pandas as pd

# Obtener variables de entorno
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Verificar si todas las variables de entorno están definidas
if not all([host, user, password]):
    raise ValueError("Faltan algunas variables de entorno (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Crear la cadena de conexión
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Crear el motor de SQLAlchemy
engine = create_engine(connection_string)

# Definir la consulta SQL para importar datos
query = f"SELECT * FROM old_raw_data_delivered"

# Importar los datos a un DataFrame de pandas
df = pd.read_sql(query, engine)

# Ruta base donde están las carpetas de años
base_path = r"C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1"

# Función para duplicar archivos
def duplicate_files(year, df_year, base_path):
    year_path = os.path.join(base_path, str(year))
    files = sorted([f for f in os.listdir(year_path) if f.endswith('.csv')])

    # Obtener los archivos existentes
    existing_files = {int(f.split('-')[1]): f for f in files}
    
    # Obtener el último archivo del año anterior
    if year > 1994:
        prev_year_path = os.path.join(base_path, str(year - 1))
        prev_files = sorted([f for f in os.listdir(prev_year_path) if f.endswith('.csv')])
        if prev_files:
            last_prev_file = prev_files[-1]
            last_prev_id_ns = int(last_prev_file.split('-')[1])
        else:
            last_prev_file = None
            last_prev_id_ns = None
    else:
        last_prev_file = None
        last_prev_id_ns = None

    # Inicializar variables para duplicación
    last_existing_file = last_prev_file
    last_existing_id_ns = last_prev_id_ns

    for index, row in df_year.iterrows():
        id_ns = row['id_ns']
        if row['delivered_1'] == 1:
            last_existing_file = existing_files[id_ns]
            last_existing_id_ns = id_ns
        else:
            # Crear nombre del nuevo archivo duplicado
            new_file_name = f"ns-{id_ns:02d}-{year}.csv"
            new_file_path = os.path.join(year_path, new_file_name)

            # Duplicar archivo
            if last_existing_file:
                if last_existing_file in existing_files.values():
                    src_file_path = os.path.join(year_path, last_existing_file)
                else:
                    src_file_path = os.path.join(prev_year_path, last_existing_file)
                shutil.copy(src_file_path, new_file_path)
                print(f"Duplicated {last_existing_file} to {new_file_name}")
            else:
                print(f"No existing file to duplicate for {new_file_name}")

# Procesar cada año
for year in range(2010, 2013):
    df_year = df[df['year'] == year]
    duplicate_files(year, df_year, base_path)


No existing file to duplicate for ns-01-2010.csv
No existing file to duplicate for ns-02-2010.csv
Duplicated ns-04-2010.csv to ns-05-2010.csv
Duplicated ns-04-2010.csv to ns-06-2010.csv
Duplicated ns-08-2010.csv to ns-09-2010.csv
Duplicated ns-08-2010.csv to ns-10-2010.csv
Duplicated ns-11-2010.csv to ns-12-2010.csv
Duplicated ns-11-2010.csv to ns-13-2010.csv
Duplicated ns-11-2010.csv to ns-14-2010.csv
Duplicated ns-15-2010.csv to ns-16-2010.csv
Duplicated ns-15-2010.csv to ns-17-2010.csv
Duplicated ns-15-2010.csv to ns-18-2010.csv
Duplicated ns-19-2010.csv to ns-20-2010.csv
Duplicated ns-19-2010.csv to ns-21-2010.csv
Duplicated ns-19-2010.csv to ns-22-2010.csv
Duplicated ns-23-2010.csv to ns-24-2010.csv
Duplicated ns-23-2010.csv to ns-25-2010.csv
Duplicated ns-23-2010.csv to ns-26-2010.csv
Duplicated ns-23-2010.csv to ns-27-2010.csv
Duplicated ns-28-2010.csv to ns-29-2010.csv
Duplicated ns-28-2010.csv to ns-30-2010.csv
Duplicated ns-31-2010.csv to ns-32-2010.csv
Duplicated ns-31-2010.